<a href="https://colab.research.google.com/github/Jlburns23/Finance/blob/main/Portfolio_performance_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# @title
# ===== CELL 1: Installation and Imports =====
!pip install yfinance plotly dash --quiet

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime
import warnings
warnings.filterwarnings('ignore')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 31.9 MB/s eta 0:00:00


In [3]:
# @title
# ===== CELL 2: Portfolio Analyzer Class Definition =====
class PortfolioAnalyzer:
    def __init__(self, tickers, shares, start_date="2025-01-01", benchmark="SPY"):
        import datetime  # Import datetime here
        self.tickers = tickers
        self.shares = shares
        self.start_date = start_date
        self.end_date = datetime.date.today()
        self.benchmark = benchmark
        self.data = None
        self.portfolio_value = None
        self.portfolio_returns = None
        self.benchmark_returns = None

    def fetch_data(self):
        """Fetch stock data with error handling"""
        try:
            # Add benchmark to tickers if not already included
            all_tickers = list(set(self.tickers + [self.benchmark]))
            self.data = yf.download(all_tickers, start=self.start_date, end=self.end_date)
            print(f"✅ Successfully downloaded data for {len(all_tickers)} tickers")
            return True
        except Exception as e:
            print(f"❌ Error fetching data: {e}")
            return False

    def calculate_portfolio_metrics(self):
        """Calculate comprehensive portfolio metrics"""
        if self.data is None:
            return None

        adj_close = self.data['Close']

        # Portfolio value over time
        portfolio_stocks = adj_close[self.tickers]
        self.portfolio_value = (portfolio_stocks * pd.Series(self.shares)).sum(axis=1)

        # Returns
        self.portfolio_returns = self.portfolio_value.pct_change().dropna()
        self.benchmark_returns = adj_close[self.benchmark].pct_change().dropna()

        # Align returns for comparison
        common_dates = self.portfolio_returns.index.intersection(self.benchmark_returns.index)
        self.portfolio_returns = self.portfolio_returns[common_dates]
        self.benchmark_returns = self.benchmark_returns[common_dates]

        # Calculate metrics
        metrics = {
            'initial_value': self.portfolio_value.iloc[0],
            'final_value': self.portfolio_value.iloc[-1],
            'total_return': (self.portfolio_value.iloc[-1] / self.portfolio_value.iloc[0] - 1) * 100,
            'annual_return': self.portfolio_returns.mean() * 252 * 100,
            'volatility': self.portfolio_returns.std() * np.sqrt(252) * 100,
            'sharpe_ratio': (self.portfolio_returns.mean() * 252) / (self.portfolio_returns.std() * np.sqrt(252)),
            'max_drawdown': self.calculate_max_drawdown(),
            'sortino_ratio': self.calculate_sortino_ratio(),
            'calmar_ratio': self.calculate_calmar_ratio(),
            'beta': self.calculate_beta(),
            'alpha': self.calculate_alpha(),
            'information_ratio': self.calculate_information_ratio()
        }

        return metrics

    def calculate_max_drawdown(self):
        """Calculate maximum drawdown"""
        cumulative = (1 + self.portfolio_returns).cumprod()
        rolling_max = cumulative.expanding().max()
        drawdown = (cumulative - rolling_max) / rolling_max
        return drawdown.min() * 100

    def calculate_sortino_ratio(self):
        """Calculate Sortino ratio (downside deviation)"""
        downside_returns = self.portfolio_returns[self.portfolio_returns < 0]
        downside_std = downside_returns.std() * np.sqrt(252)
        if downside_std == 0:
            return np.inf
        return (self.portfolio_returns.mean() * 252) / downside_std

    def calculate_calmar_ratio(self):
        """Calculate Calmar ratio"""
        max_dd = abs(self.calculate_max_drawdown() / 100)
        if max_dd == 0:
            return np.inf
        return (self.portfolio_returns.mean() * 252 * 100) / max_dd

    def calculate_beta(self):
        """Calculate portfolio beta vs benchmark"""
        covariance = np.cov(self.portfolio_returns, self.benchmark_returns)[0][1]
        benchmark_variance = np.var(self.benchmark_returns)
        return covariance / benchmark_variance

    def calculate_alpha(self):
        """Calculate Jensen's alpha"""
        risk_free_rate = 0.05  # Assume 5% risk-free rate
        portfolio_excess = self.portfolio_returns.mean() * 252 - risk_free_rate
        benchmark_excess = self.benchmark_returns.mean() * 252 - risk_free_rate
        beta = self.calculate_beta()
        return (portfolio_excess - beta * benchmark_excess) * 100

    def calculate_information_ratio(self):
        """Calculate information ratio"""
        tracking_error = (self.portfolio_returns - self.benchmark_returns).std() * np.sqrt(252)
        if tracking_error == 0:
            return 0
        return ((self.portfolio_returns.mean() - self.benchmark_returns.mean()) * 252) / tracking_error

    def get_sector_breakdown(self):
        """Get sector breakdown with error handling"""
        sector_data = []
        adj_close = self.data['Close']
        latest_prices = adj_close.iloc[-1]

        for ticker in self.tickers:
            try:
                info = yf.Ticker(ticker).info
                sector = info.get("sector", "Unknown")
                market_cap = info.get("marketCap", 0)
                value = self.shares[ticker] * latest_prices[ticker]

                sector_data.append({
                    'Ticker': ticker,
                    'Sector': sector,
                    'Shares': self.shares[ticker],
                    'Price': latest_prices[ticker],
                    'Value': value,
                    'Market_Cap': market_cap
                })
            except:
                # Fallback for ETFs or when info fails
                sector_data.append({
                    'Ticker': ticker,
                    'Sector': 'ETF/Unknown',
                    'Shares': self.shares[ticker],
                    'Price': latest_prices[ticker],
                    'Value': self.shares[ticker] * latest_prices[ticker],
                    'Market_Cap': 0
                })

        df = pd.DataFrame(sector_data)
        total_value = df['Value'].sum()
        df['Portfolio_Weight'] = (df['Value'] / total_value) * 100

        return df

    def create_comprehensive_dashboard(self):
        """Create comprehensive dashboard with multiple visualizations"""

        # Calculate cumulative returns
        portfolio_cumulative = (1 + self.portfolio_returns).cumprod()
        benchmark_cumulative = (1 + self.benchmark_returns).cumprod()

        # Create subplots with proper specs for pie chart
        fig = make_subplots(
            rows=3, cols=2,
            subplot_titles=[
                'Portfolio vs Benchmark Performance',
                'Portfolio Value Over Time',
                'Rolling Sharpe Ratio (30-day)',
                'Drawdown Analysis',
                'Monthly Returns',
                'Sector Allocation'
            ],
            specs=[[{"secondary_y": False}, {"secondary_y": False}],
                   [{"secondary_y": False}, {"secondary_y": False}],
                   [{"secondary_y": False}, {"type": "domain"}]]  # domain type for pie chart
        )

        # 1. Portfolio vs Benchmark
        fig.add_trace(
            go.Scatter(x=portfolio_cumulative.index, y=portfolio_cumulative.values,
                      name='Portfolio', line=dict(color='blue')),
            row=1, col=1
        )
        fig.add_trace(
            go.Scatter(x=benchmark_cumulative.index, y=benchmark_cumulative.values,
                      name=f'{self.benchmark} Benchmark', line=dict(color='red')),
            row=1, col=1
        )

        # 2. Portfolio Value
        fig.add_trace(
            go.Scatter(x=self.portfolio_value.index, y=self.portfolio_value.values,
                      name='Portfolio Value ($)', fill='tonexty', line=dict(color='green')),
            row=1, col=2
        )

        # 3. Rolling Sharpe Ratio
        rolling_sharpe = self.portfolio_returns.rolling(30).mean() / self.portfolio_returns.rolling(30).std() * np.sqrt(252)
        fig.add_trace(
            go.Scatter(x=rolling_sharpe.index, y=rolling_sharpe.values,
                      name='30-Day Sharpe', line=dict(color='purple')),
            row=2, col=1
        )

        # 4. Drawdown
        cumulative = (1 + self.portfolio_returns).cumprod()
        rolling_max = cumulative.expanding().max()
        drawdown = (cumulative - rolling_max) / rolling_max * 100
        fig.add_trace(
            go.Scatter(x=drawdown.index, y=drawdown.values,
                      name='Drawdown (%)', fill='tozeroy', line=dict(color='red')),
            row=2, col=2
        )

        # 5. Monthly Returns
        monthly_returns = self.portfolio_returns.groupby([
            self.portfolio_returns.index.year,
            self.portfolio_returns.index.month
        ]).apply(lambda x: (1 + x).prod() - 1) * 100

        fig.add_trace(
            go.Bar(x=monthly_returns.index.get_level_values(1), y=monthly_returns.values,
                   name='Monthly Returns (%)', marker_color='lightblue'),
            row=3, col=1
        )

        # 6. Sector Allocation (Pie Chart)
        sector_df = self.get_sector_breakdown()
        sector_summary = sector_df.groupby('Sector')['Portfolio_Weight'].sum()

        fig.add_trace(
            go.Pie(labels=sector_summary.index, values=sector_summary.values,
                   name='Sector Allocation', showlegend=False),
            row=3, col=2
        )

        fig.update_layout(height=1200, showlegend=True,
                         title_text="Portfolio Performance Dashboard")
        fig.show()

        # Also create a separate detailed sector pie chart
        sector_fig = px.pie(
            values=sector_summary.values,
            names=sector_summary.index,
            title="Detailed Sector Allocation",
            color_discrete_sequence=px.colors.qualitative.Set3
        )
        sector_fig.update_traces(textposition='inside', textinfo='percent+label')
        sector_fig.show()

        return fig

In [4]:
# @title
# ===== CELL 3: Google Sheets Integration =====
# Install required library for Google Sheets
!pip install gspread google-auth --quiet

import gspread
from google.colab import auth
from google.auth import default

def load_portfolio_from_sheets(sheet_url= 'https://docs.google.com/spreadsheets/d/1HHE2133kODTrhoJOsHTYsblieuR9PVVJSLuDc5JqL8Y/edit?usp=sharing', sheet_name='Portfolio'):
    """Load portfolio data from Google Sheets"""
    try:
        # Authenticate with Google
        auth.authenticate_user()

        # If no URL provided, use default template
        if sheet_url is None:
            print("📋 No Google Sheet URL provided. Using default portfolio...")
            # Default portfolio as fallback
            portfolio_data = {
                'AAPL': 10,
                'MSFT': 15,
                'GOOGL': 5,
                'NVDA': 8
            }
            return list(portfolio_data.keys()), portfolio_data

        # Get default credentials
        creds, _ = default()

        # Connect to Google Sheets with proper credentials
        gc = gspread.authorize(creds)

        # Open the sheet
        sheet = gc.open_by_url(sheet_url).worksheet(sheet_name)

        # Get all records
        records = sheet.get_all_records()

        # Convert to our format
        tickers = []
        shares = {}

        for record in records:
            ticker = record.get('Ticker', record.get('ticker', record.get('TICKER', ''))).upper()
            share_count = record.get('Shares', record.get('shares', record.get('SHARES', 0)))

            if ticker and share_count > 0:
                tickers.append(ticker)
                shares[ticker] = int(share_count)

        print(f"✅ Successfully loaded {len(tickers)} positions from Google Sheets")
        print(f"Portfolio: {shares}")

        return tickers, shares

    except Exception as e:
        print(f"❌ Error loading from Google Sheets: {e}")
        print("📋 Using default portfolio instead...")

        # Fallback to default portfolio
        portfolio_data = {
            'AAPL': 10,
            'MSFT': 15,
            'GOOGL': 5,
            'NVDA': 8
        }
        return list(portfolio_data.keys()), portfolio_data

# Interactive setup
print("🚀 Enhanced Portfolio Performance Dashboard")
print("=" * 50)
print("\n📊 Portfolio Data Source Options:")
print("1. Google Sheets (recommended)")
print("2. Default portfolio")

# Get Google Sheet URL from user
use_sheets = input("\nDo you want to use Google Sheets? (y/n, default=n): ").strip().lower()

if use_sheets in ['y', 'yes']:
    print("\n📋 Google Sheets Setup:")
    print("Your sheet should have columns: 'Ticker' and 'Shares'")
    print("Example:")
    print("| Ticker | Shares |")
    print("|--------|--------|")
    print("| AAPL   | 10     |")
    print("| MSFT   | 15     |")
    print("| GOOGL  | 5      |")

    sheet_url = input("\nPaste your Google Sheets URL: ").strip()
    sheet_name = input("Sheet name (default='Sheet1'): ").strip() or 'Sheet1'

    if sheet_url:
        tickers, shares = load_portfolio_from_sheets(sheet_url, sheet_name)
    else:
        print("⚠️ No URL provided, using default portfolio")
        tickers, shares = load_portfolio_from_sheets()
else:
    print("\n📋 Using default portfolio...")
    tickers, shares = load_portfolio_from_sheets()

print(f"\n✅ Final Portfolio Configuration:")
print(f"Tickers: {tickers}")
print(f"Shares: {shares}")

# Initialize analyzer
analyzer = PortfolioAnalyzer(tickers, shares, start_date="2024-01-01", benchmark="SPY")

🚀 Enhanced Portfolio Performance Dashboard

📊 Portfolio Data Source Options:
1. Google Sheets (recommended)
2. Default portfolio

Do you want to use Google Sheets? (y/n, default=n): y

📋 Google Sheets Setup:
Your sheet should have columns: 'Ticker' and 'Shares'
Example:
| Ticker | Shares |
|--------|--------|
| AAPL   | 10     |
| MSFT   | 15     |
| GOOGL  | 5      |

Paste your Google Sheets URL: 
Sheet name (default='Sheet1'): Portfolio
⚠️ No URL provided, using default portfolio
✅ Successfully loaded 6 positions from Google Sheets
Portfolio: {'AAPL': 18, 'MSFT': 12, 'GOOGL': 9, 'NVDA': 21, 'TSLA': 7, 'SPY': 8}

✅ Final Portfolio Configuration:
Tickers: ['AAPL', 'MSFT', 'GOOGL', 'NVDA', 'TSLA', 'SPY']
Shares: {'AAPL': 18, 'MSFT': 12, 'GOOGL': 9, 'NVDA': 21, 'TSLA': 7, 'SPY': 8}


In [5]:
# @title
# ===== CELL 4: Data Fetching and Analysis Execution =====
# Fetch data
if analyzer.fetch_data():
    # Calculate metrics
    metrics = analyzer.calculate_portfolio_metrics()

    if metrics:
        # Display comprehensive metrics
        print(f"\n📊 PORTFOLIO PERFORMANCE METRICS")
        print("=" * 50)
        print(f"Initial Portfolio Value: ${metrics['initial_value']:,.2f}")
        print(f"Final Portfolio Value:   ${metrics['final_value']:,.2f}")
        print(f"Total Return:           {metrics['total_return']:.2f}%")
        print(f"Annualized Return:      {metrics['annual_return']:.2f}%")
        print(f"Annualized Volatility:  {metrics['volatility']:.2f}%")
        print(f"Sharpe Ratio:           {metrics['sharpe_ratio']:.2f}")
        print(f"Sortino Ratio:          {metrics['sortino_ratio']:.2f}")
        print(f"Calmar Ratio:           {metrics['calmar_ratio']:.2f}")
        print(f"Maximum Drawdown:       {metrics['max_drawdown']:.2f}%")
        print(f"Beta (vs SPY):          {metrics['beta']:.2f}")
        print(f"Alpha (vs SPY):         {metrics['alpha']:.2f}%")
        print(f"Information Ratio:      {metrics['information_ratio']:.2f}")

    else:
        print("❌ Failed to calculate metrics")
else:
    print("❌ Failed to fetch data")

[*********************100%***********************]  6 of 6 completed


✅ Successfully downloaded data for 6 tickers

📊 PORTFOLIO PERFORMANCE METRICS
Initial Portfolio Value: $15,404.01
Final Portfolio Value:   $23,330.26
Total Return:           51.46%
Annualized Return:      28.09%
Annualized Volatility:  24.34%
Sharpe Ratio:           1.15
Sortino Ratio:          1.60
Calmar Ratio:           98.62
Maximum Drawdown:       -28.48%
Beta (vs SPY):          1.31
Alpha (vs SPY):         1.56%
Information Ratio:      0.62


In [6]:
# @title
# ===== CELL 5: Sector Analysis and Visualization =====
if 'metrics' in locals() and metrics:
    # Sector breakdown
    print(f"\n🏢 SECTOR BREAKDOWN")
    print("=" * 30)
    sector_df = analyzer.get_sector_breakdown()
    display(sector_df[['Ticker', 'Sector', 'Shares', 'Price', 'Value', 'Portfolio_Weight']])

    # Create comprehensive dashboard
    print(f"\n📈 Generating Comprehensive Dashboard...")
    dashboard = analyzer.create_comprehensive_dashboard()


🏢 SECTOR BREAKDOWN


,Ticker,Sector,Shares,Price,Value,Portfolio_Weight
0,AAPL,Technology,18,232.139999,4178.519989,17.910302
1,MSFT,Technology,12,506.690002,6080.280029,26.061776
2,GOOGL,Communication Services,9,212.910004,1916.190033,8.213325
3,NVDA,Technology,21,174.179993,3657.779846,15.678265
4,TSLA,Consumer Cyclical,7,333.869995,2337.089966,10.017419
5,SPY,Unknown,8,645.049988,5160.399902,22.118913



📈 Generating Comprehensive Dashboard...


In [7]:
# @title
# ===== CELL 6: Data Export and Summary =====
if 'metrics' in locals() and metrics:
    # Save data
    print(f"\n💾 Saving data files...")
    analyzer.portfolio_value.to_csv("portfolio_value.csv")
    analyzer.portfolio_returns.to_csv("portfolio_returns.csv")
    sector_df.to_csv("portfolio_holdings.csv", index=False)

    print("✅ Dashboard complete! Files saved.")
    print("\n📋 Files created:")
    print("- portfolio_value.csv: Daily portfolio values")
    print("- portfolio_returns.csv: Daily portfolio returns")
    print("- portfolio_holdings.csv: Detailed holdings breakdown")

else:
    print("❌ Analysis incomplete - no data to save")


💾 Saving data files...
✅ Dashboard complete! Files saved.

📋 Files created:
- portfolio_value.csv: Daily portfolio values
- portfolio_returns.csv: Daily portfolio returns
- portfolio_holdings.csv: Detailed holdings breakdown
